<h1>Connect to SQL via Langchain

In [43]:
from langchain_community.utilities import SQLDatabase

db_file = (r'files\sql-murder-mystery.db')
db = SQLDatabase.from_uri(f"sqlite:///{db_file}")
# print(db.dialect)
# print(db.get_usable_table_names())

QUERY = "SELECT * FROM person LIMIT 5"

db.run(QUERY)

"[(10000, 'Christoper Peteuil', 993845, 624, 'Bankhall Ave', 747714076), (10007, 'Kourtney Calderwood', 861794, 2791, 'Gustavus Blvd', 477972044), (10010, 'Muoi Cary', 385336, 741, 'Northwestern Dr', 828638512), (10016, 'Era Moselle', 431897, 1987, 'Wood Glade St', 614621061), (10025, 'Trena Hornby', 550890, 276, 'Daws Hill Way', 223877684)]"

In [51]:
# 
from langchain.chains import create_sql_query_chain
from langchain_community.llms.ollama import Ollama
llm = Ollama(model="llama3", base_url='http://localhost:11434')
chain = create_sql_query_chain(llm, db)

STATIC_MESSAGE = 'Answer the question verbally and add the SQL query to prove it. Make the query one line so it is easy to test. Use the simplest and most efficient query'

QUESTION = "How many people are there?"

response = chain.invoke({"question": f"{STATIC_MESSAGE} {QUESTION}"})
print(response)

Here's my answer:

Question: Answer the question verbally and add the SQL query to prove it. Make the query one line so it is easy to test. Use the simplest and most efficient query How many people are there?

Answer: There are 9 people.

SQLQuery: SELECT COUNT(*) FROM person;

Note: This query counts the number of rows in the `person` table, which represents the total number of people.


In [70]:
# TEST QUERY
QUERY = "SELECT COUNT(*) FROM person"

result = db.run(QUERY)
formatted_result = f"Liczba osób w bazie danych: {result}"
print(formatted_result)

Liczba osób w bazie danych: [(10011,)]


<h1>Connect to LLAMA3 over docker

In [41]:
from langchain_community.llms.ollama import Ollama
llm = Ollama(model="llama3", base_url='http://localhost:11434')

llm("Hello world")


'A classic!\n\n"Hello, World!" is a traditional greeting used in many programming languages to introduce a program or script. It\'s often the first output of a new program, and it\'s become a sort of iconic phrase in the world of computer programming.\n\nSo, what brings you here today? Are you exploring the world of AI, learning about language models like me, or perhaps seeking help with something else entirely? Let\'s chat!'